In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
MAT_FILE_PATH = "/content/drive/MyDrive/nyu_depth_v2_labeled (1).mat"


In [ ]:
import h5py

# Load the .mat file using h5py
with h5py.File(MAT_FILE_PATH, 'r') as file:
    data = dict(file) # Convert HDF5 dataset to dictionary format

# Now you can access the data like before
print(data.keys())

dict_keys(['#refs#', '#subsystem#', 'accelData', 'depths', 'images', 'instances', 'labels', 'names', 'namesToIds', 'rawDepthFilenames', 'rawDepths', 'rawRgbFilenames', 'sceneTypes', 'scenes'])


In [ ]:
import h5py

# Load the .mat file using h5py
with h5py.File(MAT_FILE_PATH, 'r') as file:
    # Access data assuming 'images' and 'depths' are datasets
    images = file['images'][:]  # [:] to load data into memory
    depths = file['depths'][:]

print("Images shape:", images.shape)
print("Depths shape:", depths.shape)

Images shape: (1449, 3, 640, 480)
Depths shape: (1449, 640, 480)


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import h5py

# Load the .mat file using h5py
with h5py.File(MAT_FILE_PATH, 'r') as file:
    # Access data and load into memory inside the 'with' block
    images = file['images'][:]
    depths = file['depths'][:]

# Output folders
OUTPUT_IMAGE_DIR = "/content/drive/MyDrive/nyu_small/images"
OUTPUT_DEPTH_DIR = "/content/drive/MyDrive/nyu_small/depths"

os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)
os.makedirs(OUTPUT_DEPTH_DIR, exist_ok=True)

NUM_SAMPLES = 1000

for i in tqdm(range(NUM_SAMPLES)):
    img = images[i]          # (3, 640, 480)
    depth = depths[i]        # (640, 480)

    # Transpose to (640, 480, 3)
    img = np.transpose(img, (1, 2, 0))

    # Save images and depths
    img_path = os.path.join(OUTPUT_IMAGE_DIR, f"{i:04d}.png")
    depth_path = os.path.join(OUTPUT_DEPTH_DIR, f"{i:04d}.png")

    img = np.clip(img, 0, 255).astype(np.uint8)
    depth = (depth * 1000).astype(np.uint16)   # Scale depth to millimeters if needed

    cv2.imwrite(img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    cv2.imwrite(depth_path, depth)

100%|██████████| 1000/1000 [01:00<00:00, 16.59it/s]


In [ ]:
# Check if GPU is available and set the device
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Install necessary libraries
!pip install --upgrade transformers datasets accelerate torch torchvision

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from transformers import DPTForDepthEstimation, DPTFeatureExtractor
from peft import PeftModel, LoraConfig, TaskType
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler

# Enable benchmark mode
torch.backends.cudnn.benchmark = True

# Paths
MAT_FILE_PATH = '/content/drive/MyDrive/nyu_depth_v2_labeled (1).mat'  # Not used directly here
TRAIN_DIR = "/content/drive/MyDrive/nyu_small/images"
DEPTH_DIR = "/content/drive/MyDrive/nyu_small/depths"

# Load pre-trained Intel DPT model and feature extractor
# Apply LoRA
peft_config = LoraConfig(
    task_type=TaskType.FEATURE_EXTRACTION,  # since DPT is a vision transformer-based model
    inference_mode=False,
    r=8,             # Rank of LoRA decomposition
    lora_alpha=16,   # Scaling factor
    lora_dropout=0.1, # Dropout for regularization
    target_modules=["to_q", "to_k", "to_v"], # Adjust target modules
)

# Load the base model
model = DPTForDepthEstimation.from_pretrained("Intel/dpt-large", torch_dtype=torch.float16)

# Wrap the base model with PEFT
model = PeftModel.from_pretrained(model, '/content/drive/MyDrive/fine_tuned_dpt_lora',
                                 peft_config=peft_config, torch_dtype=torch.float16)

feature_extractor = DPTFeatureExtractor.from_pretrained("Intel/dpt-large")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Show how many parameters are trainable
model.print_trainable_parameters()

# Define your custom dataset
class NYUDepthDataset(Dataset):
    def __init__(self, images_dir, depths_dir, transform=None):
        self.images_dir = images_dir
        self.depths_dir = depths_dir
        self.image_files = sorted(os.listdir(images_dir))
        self.depth_files = sorted(os.listdir(depths_dir))
        self.transform = transform
        self.to_tensor = T.ToTensor()

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = os.path.join(self.images_dir, self.image_files[idx])
        depth_path = os.path.join(self.depths_dir, self.depth_files[idx])

        image = Image.open(image_path).convert("RGB")
        depth = Image.open(depth_path)

        # Resize images
        image = T.Resize((320, 320))(image)
        image = self.to_tensor(image)

        if self.transform:
            image = self.transform(image)

        depth = np.array(depth).astype(np.float32) / 1000.0  # Convert depth to meters
        depth = torch.from_numpy(depth).unsqueeze(0)  # Add channel dimension

        return {'pixel_values': image, 'labels': depth}

# Create dataset and dataloader
dataset = NYUDepthDataset(TRAIN_DIR, DEPTH_DIR)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)  # Batch size 1 with accumulation

# Set up optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

# For mixed precision training
scaler = GradScaler()

# Training loop parameters
num_epochs = 10
accumulation_steps = 4  # Accumulate gradients over 4 iterations
loss_values = []

# Training loop
for epoch in range(num_epochs):
    model.train()
    loop = tqdm(dataloader, leave=True)
    optimizer.zero_grad()
    for idx, batch in enumerate(loop):
        pixel_values = batch['pixel_values'].to(device)
        labels = batch['labels'].to(device)

        with autocast():
            outputs = model(pixel_values=pixel_values)
            predicted_depth = outputs.predicted_depth
            loss = loss_fn(predicted_depth, labels) / accumulation_steps  # Divide loss for accumulation

        scaler.scale(loss).backward()

        if (idx + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item() * accumulation_steps)  # Multiply back for reporting
        loss_values.append(loss.item() * accumulation_steps)

# Plot training loss
plt.plot(loss_values)
plt.title("Training Loss")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.show()

# Save LoRA adapted model
model.save_pretrained("/content/drive/MyDrive/fine_tuned_dpt_lora")

print("Training completed and model saved!")